In [1]:
import numpy as np 
import matplotlib
import matplotlib.pyplot as plt
import random
import time
import cvxpy as cp
import pandas as pd
from sklearn.model_selection import train_test_split
import collections

/home/haskari/.conda/envs/ML/lib/python3.9/site-packages/sklearn/utils/multiclass.py:13: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  from scipy.sparse.base import spmatrix


In [3]:
TRAIN_DATA = pd.read_csv('transfer_train-1.csv')
TEST_DATA=pd.read_csv('transfer_test-1.csv')

TRAIN_DATA_W_LABELS = TRAIN_DATA[(TRAIN_DATA['Digit']==1) | (TRAIN_DATA['Digit']==7)]

D = 8 

ONLY_LABELS = TRAIN_DATA_W_LABELS[['Digit']]


ONLY_LABELS.loc[ONLY_LABELS['Digit'] == 7, 'Digit'] = -1


TRAIN_DATA_NO_LABEL = TRAIN_DATA_W_LABELS.drop('Digit', axis=1)

# Train_data_normalized=TRAIN_DATA_NO_LABEL.copy()

# for column in Train_data_normalized.columns:
#     Train_data_normalized[column] = (Train_data_normalized[column] - Train_data_normalized[column].min()) / (Train_data_normalized[column].max() - Train_data_normalized[column].min()) 

TEST_DATA_W_LABELS = TEST_DATA[(TEST_DATA['Digit']==1) | (TEST_DATA['Digit']==7)]

ONLY_LABELS_TEST = TEST_DATA_W_LABELS[['Digit']]

ONLY_LABELS_TEST.loc[ONLY_LABELS_TEST['Digit'] == 7, 'Digit'] = -1

TEST_DATA_NO_LABEL = TEST_DATA_W_LABELS.drop('Digit', axis=1)

# Test_data_normalized=TEST_DATA_NO_LABEL.copy()

# for column in Test_data_normalized.columns:
#     Test_data_normalized[column] = (Test_data_normalized[column] - Test_data_normalized[column].min()) / (Test_data_normalized[column].max() - Test_data_normalized[column].min())  

/tmp/ipykernel_832013/3200448627.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ONLY_LABELS.loc[ONLY_LABELS['Digit'] == 7, 'Digit'] = -1
/tmp/ipykernel_832013/3200448627.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ONLY_LABELS_TEST.loc[ONLY_LABELS_TEST['Digit'] == 7, 'Digit'] = -1


In [4]:
X_train_np=TRAIN_DATA_NO_LABEL.to_numpy()
print(X_train_np)
print(X_train_np.shape)
y_train_np=ONLY_LABELS.to_numpy()
print(y_train_np.shape)
X_test=TEST_DATA_NO_LABEL.to_numpy()
y_test=ONLY_LABELS_TEST.to_numpy()

[[ 73  84 100 100  88  75  81  50]
 [ 48  78  41  36  21   0   0   3]
 [ 42  83  79 100 100  98  81  66]
 [ 81 100 100  95  81  70  57  45]
 [ 48  49  76  73 100 100  94  73]
 [ 90  90  90  48  76   6  55   0]
 [100  79  83  44  67   8  45   0]
 [ 65 100 100 100  66  76  47  49]
 [ 44  59  39  20  29   0   0  31]
 [ 35  51  32   0  13  13  22  32]]
(10, 8)
(10, 1)


In [5]:
train_data_qty = len(X_train_np)
print(train_data_qty)

10


In [9]:
C = 2

w = cp.Variable((D, 1))

b = cp.Variable()

epsilon = cp.Variable((train_data_qty, 1))

objective = cp.Minimize(cp.sum(cp.square(w))*0.5 + cp.sum(cp.square(epsilon)*C))

#found support vectors from 1vs9
sup_vecs_from_1vs9 = np.array([[ 98,  64,  69, 100,  58,  69, 100,  61],
       [ 71,  91, 100, 100,  82,  74,  68,  48],
       [ 66,  82, 100, 100,  77,  76,  55,  51],
       [ 96, 100, 100,  84,  94,  60,  88,  35],
       [100, 100,  89,  75,  79,  49,  76,  24]])

#found labels of support vectors from 1vs9
labels_for_sup_vec_1vs9 = np.array([[-1], [ 1], [ 1], [ 1], [ 1]])

constraints = [cp.multiply(y_train_np, (X_train_np @ (w) + b)) >= 1 - epsilon, cp.multiply(labels_for_sup_vec_1vs9, (sup_vecs_from_1vs9 @ w + b)) >= 1,epsilon >= 0]

prob = cp.Problem(objective, constraints)
prob.solve()
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var w = {}, b = {}".format(w.value, b.value))

status: optimal
optimal value 0.07047557811658729
optimal var w = [[-0.02827472]
 [-0.01413494]
 [-0.23735641]
 [ 0.0651211 ]
 [ 0.22420957]
 [ 0.09330395]
 [-0.05573119]
 [-0.12866736]], b = 6.52074126118702


In [10]:
Xt = X_test

#Xt.shape

yt = y_test

#yt.shape

predictions = Xt.dot(w.value) + b.value

predictions[predictions<0] = -1

predictions[predictions>=0] = 1

count_wrong_predictions = 0
for i in range(len(predictions)):
    if (yt[i] != predictions[i]):
        count_wrong_predictions+=1
        
#count_wrong_predictions

correct_predictions = (len(predictions) - count_wrong_predictions)
#correct_predictions

accuracy = correct_predictions/len(predictions)

print("Accuracy =",accuracy*100,"%")

Accuracy = 78.91246684350133 %
